In [116]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [117]:
# load the dataset
data=pd.read_csv("D:/ann classification\Churn_Modelling.csv")   # \ann -- python will take this as bell character and convert this \ann as \a convert this like \x07 and the file name will be interpreted...so it cannot read the file...so to solve this give /ann or \\ann
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [118]:
# problem statement-based on some feature we are going to predict whether the customer is going to exit the bank or not- for that we need credit score,geography,gender,age,tenure,balance  etc
# preprocess the data
#drop irrelevant features
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [119]:
## encoding categorical variable
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

# label encoder = that col has only 2 features male and female so 0,1

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [120]:
# for feature having more than 2 categories--- we use ohe
# one hot encode
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [121]:
# actually we failed to convert this into sparse matrix in new version so we are giving to array function to convert this into sparse matrix
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [122]:
# to find how many features are there in that geography col
# converting this into numpy array
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [123]:
# this one and sparse matrix one (geo encoder) converting that into the form of dataframe
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [124]:
# combine one hot encoded column with original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [125]:
# this encodings are going to be used later ...so we are converting this into pickle file...so that we can use this again in later stages
with open('label_encoder_gender.pkl','wb')as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb')as file:    
    pickle.dump(onehot_encoder_geo,file)
    


In [126]:
# divide the datatset into dependent and independent features
# exit-dependent

x=data.drop('Exited',axis=1)
y=data['Exited']

# split the data in training and testing sets


x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

## scale donw these feature

scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [127]:
x_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [128]:
# save scaler form in pickle file
with open('scaler.pkl','wb')as file:
    pickle.dump(scaler,file)


### ANN implementation -- we call ann as sequential model 

In [129]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [130]:
## build our ann model-- we have created from i/p to o/p
model=Sequential([Dense(64,activation='relu',input_shape=(x_train.shape[1],)),# x_train.shape--> this is only for hl1 it is getting connected to some inputs over there
                  Dense(32,activation='relu'),## HL2
                  Dense(1,activation='sigmoid') ## output layer
                 ])

In [131]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                832       
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [132]:
# method 2 to do optimization
# either we can give like this by using a variable or like above one by giving directly
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss


In [133]:
# next to do fwd and bwd propagation-- we have to compile
# this is method 1

model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])  

# for binary classification--binary cross entrophy,for multiclass do sparse cross entrophy or sparse matrix
# there are other ways to put values and do optimization

In [134]:
# set up the tensor board -- to capture and visualize the logs

from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1) # -- we are calling tensorboard here to use for this we have to import first

# datetime (first one) → refers to the module
# datetime.datetime → refers to the datetime class inside that module
# tensorboard-- to visualise all your logs that you have, while training your entire model

In [135]:
# set up early stopping-when we are training for neural networks we can train it for any number of neural networks
# we have planned to do 100 epochs per batch-- but by 5 epoch itself we didint see any chnages in loss value then ,
# we can do early stopping since there are not any major changes ...stop by 5 epoch itself

early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

''' What restore_best_weights=True means:
It tells Keras:
"After stopping, roll back the model weights to the best epoch (where val_loss or val_accuracy was best)."
Without it, your model may end up keeping weights from the last training epoch, which might not be the best!
if false-- it will take the last epoch value which is worse '''


' What restore_best_weights=True means:\nIt tells Keras:\n"After stopping, roll back the model weights to the best epoch (where val_loss or val_accuracy was best)."\nWithout it, your model may end up keeping weights from the last training epoch, which might not be the best!\nif false-- it will take the last epoch value which is worse '

In [136]:
# train the model
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
    )

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.3922 - accuracy: 0.8350 - val_loss: 0.3847 - val_accuracy: 0.8360
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3536 - accuracy: 0.8575 - val_loss: 0.3454 - val_accuracy: 0.8615
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3503 - accuracy: 0.8577 - val_loss: 0.3429 - val_accuracy: 0.8575
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3449 - accuracy: 0.8595 - val_loss: 0.3398 - val_accuracy: 0.8615
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3410 - accuracy: 0.8604 - val_loss: 0.3465 - val_accuracy: 0.8550
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3355 - accuracy: 0.8633 - val_loss: 0.3397 - val_accuracy: 0.8610
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3342 - accuracy: 0.8659 - val_loss: 0.3491 - val_accuracy: 0.8545

In [137]:
model.save('model.h5')
# h5 is compatible with keras

d:\ann classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [138]:
pip install TensorBoard

Note: you may need to restart the kernel to use updated packages.


In [139]:
# load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [140]:
%tensorboard --logdir logs/fit20250423-221444/train/
%tensorboard --logdir logs/fit20250423-221444/validation/

Reusing TensorBoard on port 6009 (pid 3756), started 7 days, 23:29:47 ago. (Use '!kill 3756' to kill it.)

Reusing TensorBoard on port 6010 (pid 9572), started 7 days, 23:29:36 ago. (Use '!kill 9572' to kill it.)

In [141]:
# load the pickle file
